In [1]:
#Импортируем библиотеки
import pandas as pd
from datetime import datetime
import math
import numpy as np

In [2]:
#импортируем данные 
data = pd.read_csv('casc-resto.csv', sep=';', parse_dates = ['RKDate'], decimal = ",")
data

,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage
0,2898197,391,2017-07-16,Москва и Московская область,TGI FRIDAYS,NON ALCOHOL,2,2.00,2.00
1,2903215,43,2015-04-07,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.00,5.00
2,2748887,43,2015-05-22,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.00,5.00
3,2862077,46,2015-03-05,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.00,5.00
4,2862077,46,2015-03-11,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.00,5.00
...,...,...,...,...,...,...,...,...,...
882217,2811717,922,2016-01-07,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,57.67,57.67
882218,2811717,922,2016-01-07,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,57.67,57.67
882219,2811717,922,2016-01-07,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,57.67,57.67
882220,2811717,922,2016-01-07,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,57.67,57.67


In [3]:
#Проверим типы  столбцов (RKDate)
print (data.dtypes)

CustomerID                       int64
Restaurant                       int64
RKDate                  datetime64[ns]
RegionName                      object
BrandsNames                     object
DishCategoryName                object
Quantity                         int64
SummBasic                      float64
SummAfterPointsUsage           float64
dtype: object


In [4]:
#почистим данные от пустых значений
data = data.dropna()
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 881608 entries, 0 to 882221
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   CustomerID            881608 non-null  int64         
 1   Restaurant            881608 non-null  int64         
 2   RKDate                881608 non-null  datetime64[ns]
 3   RegionName            881608 non-null  object        
 4   BrandsNames           881608 non-null  object        
 5   DishCategoryName      881608 non-null  object        
 6   Quantity              881608 non-null  int64         
 7   SummBasic             881608 non-null  float64       
 8   SummAfterPointsUsage  881608 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 67.3+ MB
None


In [5]:
#изменим отрицательные значения
data = data[data.SummAfterPointsUsage > 0]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 881607 entries, 0 to 882221
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   CustomerID            881607 non-null  int64         
 1   Restaurant            881607 non-null  int64         
 2   RKDate                881607 non-null  datetime64[ns]
 3   RegionName            881607 non-null  object        
 4   BrandsNames           881607 non-null  object        
 5   DishCategoryName      881607 non-null  object        
 6   Quantity              881607 non-null  int64         
 7   SummBasic             881607 non-null  float64       
 8   SummAfterPointsUsage  881607 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 67.3+ MB


In [6]:
#Задание: зафикисировать дату 2017-07-01 как дату, на которую считаются характеристики клиентов
#также зафиксируем дату 31 декабря 2017 для дальнейших расчетов
july_first = datetime(2017, 7, 1) 
december_31 = datetime(2017, 12, 31) 
print(july_first)
print (december_31) 

2017-07-01 00:00:00
2017-12-31 00:00:00


In [7]:
# функция которая вычисляет параметры
def fun1(df):
    #Для каждого клиента определить, совершал ли он визит в период между 2017-07-01 и 2017-12-31 включительно. 
    #Полученный флаг является зависимой переменной Y, которую требуется предсказывать. назоввем ее Is_visited
    res = 0
    for date in df['RKDate']:
        if date >= july_first and date <= december_31:
            res = 1
            break
    df['Is_visited'] = res
    # Даты когда посещали клиенты до фикс. даты
    poseshenia = df[df['RKDate'] < july_first].RKDate
    if poseshenia.size == 0:
        df['Recency'] = 0
        df['Frequency'] = 0
        df['MonetaryValue'] = 0
        
    else:
        #Расчитать Recency (количество дней до последнего визита клиента перед зафиксированной датой 2017-07-01)
        df['Recency'] = (july_first - poseshenia.max()).days
        #Frequency (средняя частота походов клиентов на дату 2017-07-01
        #измеряется как среднее количество визитов в месяц [месяц полагать равным 30 дням] за время между 2017-07-01 и первым визитом клиента)
        df['Frequency'] = poseshenia.size / math.ceil((july_first - poseshenia.min()).days / 30)
        # Monetary Value (средний чек клиента по всем покупкам до 2017-07-01)
        df['MonetaryValue'] = df['SummAfterPointsUsage'].sum() / poseshenia.unique().size
        
    return df

# Вычислить указанные в задаче параметры
data_data = data.groupby('CustomerID').apply(fun1)
data_data = data_data.reset_index()
data_data

,index,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage,Is_visited,Recency,Frequency,MonetaryValue
0,0,2898197,391,2017-07-16,Москва и Московская область,TGI FRIDAYS,NON ALCOHOL,2,2.00,2.00,1,64,4.678571,4776.055556
1,1,2903215,43,2015-04-07,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.00,5.00,0,367,1.714286,1697.875000
2,2,2748887,43,2015-05-22,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.00,5.00,1,23,7.033333,1339.280000
3,3,2862077,46,2015-03-05,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.00,5.00,0,761,5.000000,565.000000
4,4,2862077,46,2015-03-11,Москва и Московская область,IL Патио,NON ALCOHOL,1,5.00,5.00,0,761,5.000000,565.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881602,882217,2811717,922,2016-01-07,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,57.67,57.67,0,286,14.250000,3017.489318
881603,882218,2811717,922,2016-01-07,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,57.67,57.67,0,286,14.250000,3017.489318
881604,882219,2811717,922,2016-01-07,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,57.67,57.67,0,286,14.250000,3017.489318
881605,882220,2811717,922,2016-01-07,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,57.67,57.67,0,286,14.250000,3017.489318


In [8]:
#импортируем данные из второго файла с полом и возраст
data_d = pd.read_csv('CASC_Constant.csv', sep=',', error_bad_lines=False)
data_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   ActivationDate   10000 non-null  object 
 2   Age              9953 non-null   float64
 3   Sex              8874 non-null   object 
 4   SubscribedEmail  10000 non-null  bool   
 5   SubscribedPush   10000 non-null  bool   
dtypes: bool(2), float64(1), int64(1), object(2)
memory usage: 332.2+ KB


In [9]:
#перевод пола в число (жен - 1; муж - 0)
data_d = pd.get_dummies(data_d, columns=['Sex'], prefix=['Sex'])


In [10]:
# заполнение пропусков в возрасте
age_mean = data_d['Age'].mean()
data_d['Age'] = data_d['Age'].fillna(age_mean)
data_d.info()
data_d

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       10000 non-null  int64  
 1   ActivationDate   10000 non-null  object 
 2   Age              10000 non-null  float64
 3   SubscribedEmail  10000 non-null  bool   
 4   SubscribedPush   10000 non-null  bool   
 5   Sex_Female       10000 non-null  uint8  
 6   Sex_Male         10000 non-null  uint8  
dtypes: bool(2), float64(1), int64(1), object(1), uint8(2)
memory usage: 273.6+ KB


,CustomerId,ActivationDate,Age,SubscribedEmail,SubscribedPush,Sex_Female,Sex_Male
0,2728183,2015-01-01,21.0,False,True,1,0
1,2728198,2015-01-01,21.0,True,True,1,0
2,2728306,2015-01-01,21.0,True,True,1,0
3,2728178,2015-01-01,22.0,True,True,0,1
4,2728322,2015-01-01,22.0,True,True,0,1
...,...,...,...,...,...,...,...
9995,2912655,2015-03-31,49.0,True,True,1,0
9996,2912502,2015-03-31,52.0,True,True,0,1
9997,2912767,2015-03-31,52.0,True,False,0,1
9998,2912383,2015-03-31,60.0,False,False,0,1


In [11]:
#Объединение таблиц 
import pandas as pd
data_common = pd.merge(left=data_data, right=data_d, left_on='CustomerID', right_on='CustomerId')
data_common.info()
data_common

<class 'pandas.core.frame.DataFrame'>
Int64Index: 881607 entries, 0 to 881606
Data columns (total 21 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   index                 881607 non-null  int64         
 1   CustomerID            881607 non-null  int64         
 2   Restaurant            881607 non-null  int64         
 3   RKDate                881607 non-null  datetime64[ns]
 4   RegionName            881607 non-null  object        
 5   BrandsNames           881607 non-null  object        
 6   DishCategoryName      881607 non-null  object        
 7   Quantity              881607 non-null  int64         
 8   SummBasic             881607 non-null  float64       
 9   SummAfterPointsUsage  881607 non-null  float64       
 10  Is_visited            881607 non-null  int64         
 11  Recency               881607 non-null  int64         
 12  Frequency             881607 non-null  float64       
 13 

,index,CustomerID,Restaurant,RKDate,RegionName,BrandsNames,DishCategoryName,Quantity,SummBasic,SummAfterPointsUsage,...,Recency,Frequency,MonetaryValue,CustomerId,ActivationDate,Age,SubscribedEmail,SubscribedPush,Sex_Female,Sex_Male
0,0,2898197,391,2017-07-16,Москва и Московская область,TGI FRIDAYS,NON ALCOHOL,2,2.0,2.00,...,64,4.678571,4776.055556,2898197,2015-03-21,33.0,False,False,1,0
1,30814,2898197,921,2015-05-23,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,30.0,30.00,...,64,4.678571,4776.055556,2898197,2015-03-21,33.0,False,False,1,0
2,125923,2898197,921,2015-05-20,Москва и Московская область,"IL Патио,Планета Суши",ALCOHOL,1,160.0,125.42,...,64,4.678571,4776.055556,2898197,2015-03-21,33.0,False,False,1,0
3,125932,2898197,921,2015-08-28,Москва и Московская область,"IL Патио,Планета Суши",ALCOHOL,1,160.0,135.36,...,64,4.678571,4776.055556,2898197,2015-03-21,33.0,False,False,1,0
4,154879,2898197,923,2017-03-04,Москва и Московская область,"IL Патио,Планета Суши",FOOD,1,865.0,707.28,...,64,4.678571,4776.055556,2898197,2015-03-21,33.0,False,False,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881602,837033,2744732,934,2015-03-30,Москва и Московская область,"IL Патио,Планета Суши",FOOD,2,540.0,540.00,...,501,6.933333,607.125000,2744732,2015-01-15,50.0,False,False,0,1
881603,837035,2744732,934,2015-04-09,Москва и Московская область,"IL Патио,Планета Суши",FOOD,2,540.0,540.00,...,501,6.933333,607.125000,2744732,2015-01-15,50.0,False,False,0,1
881604,837037,2744732,934,2015-04-27,Москва и Московская область,"IL Патио,Планета Суши",FOOD,2,540.0,540.00,...,501,6.933333,607.125000,2744732,2015-01-15,50.0,False,False,0,1
881605,837041,2744732,934,2015-10-02,Москва и Московская область,"IL Патио,Планета Суши",FOOD,2,540.0,540.00,...,501,6.933333,607.125000,2744732,2015-01-15,50.0,False,False,0,1


In [12]:
#импортр библиотек для регрессии 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [13]:
# подготовить данные к регрессии
X = data_common[['Recency', 'Frequency', 'MonetaryValue', 'Age', 'Sex_Female']].to_numpy()
Y = data_common['Is_visited'].to_numpy()

In [14]:
# Создание и обучение логистической регрессии
# Разбиение данных на обучающее и тестовое подмножества
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state=42)
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [15]:
y_predicted = logistic_regression.predict(X_test)
print(confusion_matrix(y_test, y_predicted))
print(classification_report(y_test, y_predicted))

[[ 33203  26090]
 [  8090 108939]]
              precision    recall  f1-score   support

           0       0.80      0.56      0.66     59293
           1       0.81      0.93      0.86    117029

    accuracy                           0.81    176322
   macro avg       0.81      0.75      0.76    176322
weighted avg       0.81      0.81      0.80    176322

